**1. ʟɪʙʀᴀʀɪᴇꜱ ɪᴍᴘᴏʀᴛɪɴɢ**

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

**2. ᴅᴀᴛᴀ ᴄᴏʟʟᴇᴄᴛɪᴏɴꜱ**

In [4]:
import pandas as pd

#importing dataset
data = pd.read_csv('/content/drive/MyDrive/Twitter_Data.csv')
data.head(20)


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
5,28b57f3990,http://www.dothebouncy.com/smf - some shameles...,http://www.dothebouncy.com/smf - some shameles...,neutral
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive
7,50e14c0bb8,Soooo high,Soooo high,neutral
8,e050245fbd,Both of you,Both of you,neutral
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive


**3. ᴅᴀᴛᴀ ᴄʟᴇᴀɴɪɴɢ**

In [5]:
#droping coulmns
data1 = data.drop(['textID','text'], axis=1)
data1.head()

,selected_text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD,negative
2,bullying me,negative
3,leave me alone,negative
4,"Sons of ****,",negative


In [6]:
data1.isnull().sum()

,0
selected_text,1
sentiment,0


In [7]:
#fill using mode selected_text
data1['selected_text'] = data1['selected_text'].fillna(data1['selected_text'].mode()[0])
data1.isnull().sum()

,0
selected_text,0
sentiment,0


In [8]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove special characters and digits
    text = re.sub(r'\d+', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in STOPWORDS])
    return text
    import re

def clean_text(text):
    # Remove punctuation and symbols
    text = re.sub(r'[^\w\s]', '', text)
    # Remove common numbers
    text = re.sub(r'\b\d+\b', '', text)
    # Remove underscores and replace with spaces
    text = re.sub(r'_', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Example usage
text = "Hello, world! This is an example_text with numbers 123 and special@characters."
cleaned_text = clean_text(text)
print(cleaned_text)


# Apply the cleaning function to the 'text' column
data1['cleaned_text'] = data1['selected_text'].apply(clean_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


hello world this is an example text with numbers and specialcharacters


In [9]:
data1.head(30)

,selected_text,sentiment,cleaned_text
0,"I`d have responded, if I were going",neutral,id have responded if i were going
1,Sooo SAD,negative,sooo sad
2,bullying me,negative,bullying me
3,leave me alone,negative,leave me alone
4,"Sons of ****,",negative,sons of
5,http://www.dothebouncy.com/smf - some shameles...,neutral,httpwwwdothebouncycomsmf some shameless pluggi...
6,fun,positive,fun
7,Soooo high,neutral,soooo high
8,Both of you,neutral,both of you
9,Wow... u just became cooler.,positive,wow u just became cooler


In [10]:
# Example: Reorder columns manually
data1 = data1[['cleaned_text', 'sentiment', 'selected_text']]
data1.drop(['selected_text'], axis=1, inplace=True)
data1.head(10)


<ipython-input-10-b72b38d361ad>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.drop(['selected_text'], axis=1, inplace=True)


,cleaned_text,sentiment
0,id have responded if i were going,neutral
1,sooo sad,negative
2,bullying me,negative
3,leave me alone,negative
4,sons of,negative
5,httpwwwdothebouncycomsmf some shameless pluggi...,neutral
6,fun,positive
7,soooo high,neutral
8,both of you,neutral
9,wow u just became cooler,positive


**4.ᴛᴏᴋᴇɴɪᴢᴀᴛɪᴏɴ ᴀɴᴅ ᴘᴀᴅᴅɪɴɢ**

In [11]:
# Tokenization and Padding
MAX_VOCAB_SIZE = 5000
MAX_SEQUENCE_LENGTH = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token='<OOV>')
tokenizer.fit_on_texts(data1['cleaned_text'].values)

X = tokenizer.texts_to_sequences(data1['cleaned_text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

# Encoding the Labels
le = LabelEncoder()
y = le.fit_transform(data1['sentiment'])
y = to_categorical(y)



**5. ᴇɴᴄᴏᴅɪɴɢ ᴛʜᴇ ʟᴀʙᴇʟꜱ**

In [68]:
from tensorflow.keras.utils import to_categorical

# Number of classes
num_classes = 3

# Convert integer labels to one-hot encoding
y_train_one_hot = to_categorical(y_train, num_classes=num_classes)
y_test_one_hot = to_categorical(y_test, num_classes=num_classes)

print("One-hot encoded y_train shape:", y_train_one_hot.shape)
print("One-hot encoded y_test shape:", y_test_one_hot.shape)



One-hot encoded y_train shape: (21984, 3)
One-hot encoded y_test shape: (5497, 3)


**6.ᴅᴀᴛᴀ ꜱᴘʟɪᴛɪɴɢ ꜰᴏʀ ᴛʀᴀɪɴɪɴɢ ᴀɴᴅ ꜱᴘʟɪᴛɪɴɢ ꜱᴇᴛꜱ**

In [69]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**7.ᴍᴏᴅᴇʟ ᴛʀᴀɪɴɪɴɢ ᴠᴇʀꜱɪᴏɴ 1.0**

In [70]:
print(y_train.shape)


(21984, 3)


In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense

# Define the LSTM model
EMBEDDING_DIM = 100
MAX_VOCAB_SIZE = 10000  # Example value; adjust as needed
MAX_SEQUENCE_LENGTH = 100  # Example value; adjust as needed

model1 = Sequential()
model1.add(Embedding(MAX_VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model1.add(SpatialDropout1D(0.2))
model1.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model1.add(Dense(num_classes, activation='softmax'))  # Number of classes for one-hot encoding

model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



# Train the Model
BATCH_SIZE = 64
EPOCHS = 5

history = model1.fit(X_train, y_train_one_hot, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test_one_hot), verbose=2)



Epoch 1/5
344/344 - 63s - 183ms/step - accuracy: 0.7395 - loss: 0.6348 - val_accuracy: 0.8355 - val_loss: 0.4498
Epoch 2/5
344/344 - 82s - 239ms/step - accuracy: 0.8463 - loss: 0.4069 - val_accuracy: 0.8388 - val_loss: 0.4287
Epoch 3/5
344/344 - 81s - 237ms/step - accuracy: 0.8694 - loss: 0.3491 - val_accuracy: 0.8410 - val_loss: 0.4331
Epoch 4/5
344/344 - 63s - 183ms/step - accuracy: 0.8841 - loss: 0.3111 - val_accuracy: 0.8459 - val_loss: 0.4313
Epoch 5/5
344/344 - 61s - 177ms/step - accuracy: 0.8933 - loss: 0.2851 - val_accuracy: 0.8341 - val_loss: 0.4604


**ᴍᴏᴅᴇʟ ᴇᴠᴀʟᴜᴀᴛɪᴏɴ ᴠᴇʀꜱɪᴏɴ 1.0**

In [37]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Predict class probabilities for X_test
y_pred_probs = model1.predict(X_test)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test_one_hot, axis=1)  # Ensure y_test is in one-hot encoded format

# Print classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['positive', 'negative', 'neutral']))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))



172/172 ━━━━━━━━━━━━━━━━━━━━ 8s 44ms/step
Classification Report:
              precision    recall  f1-score   support

    positive       0.78      0.84      0.81      1562
    negative       0.83      0.83      0.83      2230
     neutral       0.89      0.83      0.86      1705

    accuracy                           0.83      5497
   macro avg       0.84      0.83      0.83      5497
weighted avg       0.84      0.83      0.83      5497

Confusion Matrix:
[[1319  196   47]
 [ 244 1855  131]
 [ 122  172 1411]]


**ᴍᴏᴅᴇʟ ᴘᴇʀᴅɪᴄᴀᴛɪᴏɴ ᴠᴇʀꜱɪᴏɴ 1.0**

In [35]:
#Predicting sample test
sample_text = ["dont force"]
sample_text = tokenizer.texts_to_sequences(sample_text)
sample_text = pad_sequences(sample_text, maxlen=MAX_SEQUENCE_LENGTH)

prediction = model1.predict(sample_text)
print("Prediction:", prediction)
print("Predicted label:", le.inverse_transform(np.argmax(prediction, axis=1)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Prediction: [[0.93545836 0.02923337 0.03530821]]
Predicted label: ['negative']


**ᴍᴏᴅᴇʟ ᴛʀᴀɪɴɪɴɢ ᴠᴇʀꜱɪᴏɴ 2.0**

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense

# Define the LSTM model
EMBEDDING_DIM = 100
MAX_VOCAB_SIZE = 10000  # Example value; adjust as needed
MAX_SEQUENCE_LENGTH = 100  # Example value; adjust as needed

model2 = Sequential()
model2.add(Embedding(MAX_VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model2.add(SpatialDropout1D(0.2))
model2.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(num_classes, activation='softmax'))  # Number of classes for one-hot encoding

model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



# Train the Model
BATCH_SIZE = 64
EPOCHS = 6

history = model2.fit(X_train, y_train_one_hot, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test_one_hot), verbose=2)



Epoch 1/6


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


344/344 - 69s - 199ms/step - accuracy: 0.7354 - loss: 0.6387 - val_accuracy: 0.8366 - val_loss: 0.4488
Epoch 2/6
344/344 - 90s - 263ms/step - accuracy: 0.8473 - loss: 0.4049 - val_accuracy: 0.8456 - val_loss: 0.4258
Epoch 3/6
344/344 - 61s - 177ms/step - accuracy: 0.8706 - loss: 0.3448 - val_accuracy: 0.8450 - val_loss: 0.4281
Epoch 4/6
344/344 - 81s - 235ms/step - accuracy: 0.8857 - loss: 0.3126 - val_accuracy: 0.8392 - val_loss: 0.4386
Epoch 5/6
344/344 - 83s - 242ms/step - accuracy: 0.8934 - loss: 0.2882 - val_accuracy: 0.8405 - val_loss: 0.4481
Epoch 6/6
344/344 - 81s - 236ms/step - accuracy: 0.9000 - loss: 0.2692 - val_accuracy: 0.8368 - val_loss: 0.4653


**ᴍᴏᴅᴇʟ ᴇᴠᴀʟᴜᴀᴛɪᴏɴ ᴠᴇʀꜱɪᴏɴ 2.0**

In [41]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Predict class probabilities for X_test
y_pred_probs = model2.predict(X_test)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test_one_hot, axis=1)  # Ensure y_test is in one-hot encoded format

# Print classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['positive', 'negative', 'neutral']))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))



172/172 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step
Classification Report:
              precision    recall  f1-score   support

    positive       0.80      0.83      0.82      1562
    negative       0.82      0.86      0.84      2230
     neutral       0.91      0.81      0.86      1705

    accuracy                           0.84      5497
   macro avg       0.84      0.83      0.84      5497
weighted avg       0.84      0.84      0.84      5497

Confusion Matrix:
[[1304  221   37]
 [ 217 1908  105]
 [ 116  201 1388]]


**ᴍᴏᴅᴇʟ ᴘᴇʀᴅɪᴄᴀᴛɪᴏɴ ᴠᴇʀꜱɪᴏɴ 2.0**

In [42]:

sample_text = ["I love this product!"]
sample_text = tokenizer.texts_to_sequences(sample_text)
sample_text = pad_sequences(sample_text, maxlen=MAX_SEQUENCE_LENGTH)

prediction = model2.predict(sample_text)
print("Prediction:", prediction)
print("Predicted label:", le.inverse_transform(np.argmax(prediction, axis=1)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Prediction: [[0.00448107 0.12759176 0.86792713]]
Predicted label: ['positive']


**ᴍᴏᴅᴇʟ ᴛʀᴀɪɴɪɴɢ ᴠᴇʀꜱꜱɪᴏɴ 3.0**

In [71]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense

# Define the LSTM model
EMBEDDING_DIM = 100
MAX_VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 100

model3 = Sequential()
model3.add(Embedding(MAX_VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model3.add(SpatialDropout1D(0.2))
model3.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model3.add(Dense(num_classes, activation='softmax'))  # Number of classes for one-hot encoding

model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the Model
BATCH_SIZE = 64
EPOCHS = 7

history = model3.fit(X_train, y_train_one_hot, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test_one_hot), verbose=2)


Epoch 1/7


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


344/344 - 66s - 191ms/step - accuracy: 0.7318 - loss: 0.6389 - val_accuracy: 0.8350 - val_loss: 0.4486
Epoch 2/7
344/344 - 79s - 230ms/step - accuracy: 0.8483 - loss: 0.4034 - val_accuracy: 0.8395 - val_loss: 0.4367
Epoch 3/7
344/344 - 83s - 241ms/step - accuracy: 0.8717 - loss: 0.3457 - val_accuracy: 0.8436 - val_loss: 0.4261
Epoch 4/7
344/344 - 82s - 239ms/step - accuracy: 0.8836 - loss: 0.3113 - val_accuracy: 0.8479 - val_loss: 0.4349
Epoch 5/7
344/344 - 83s - 240ms/step - accuracy: 0.8966 - loss: 0.2841 - val_accuracy: 0.8348 - val_loss: 0.4466
Epoch 6/7
344/344 - 81s - 235ms/step - accuracy: 0.9023 - loss: 0.2668 - val_accuracy: 0.8343 - val_loss: 0.4609
Epoch 7/7
344/344 - 84s - 243ms/step - accuracy: 0.9064 - loss: 0.2524 - val_accuracy: 0.8346 - val_loss: 0.4727


**ᴍᴏᴅᴇʟ ᴇᴠᴀʟᴜᴇᴀᴛɪᴏɴ ᴠᴇʀꜱɪᴏɴ 3.0**

In [45]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Predict class probabilities for X_test
y_pred_probs = model3.predict(X_test)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test_one_hot, axis=1)  # Ensure y_test is in one-hot encoded format

# Print classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['positive', 'negative', 'neutral']))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

172/172 ━━━━━━━━━━━━━━━━━━━━ 7s 43ms/step
Classification Report:
              precision    recall  f1-score   support

    positive       0.81      0.82      0.82      1562
    negative       0.83      0.85      0.84      2230
     neutral       0.88      0.84      0.86      1705

    accuracy                           0.84      5497
   macro avg       0.84      0.84      0.84      5497
weighted avg       0.84      0.84      0.84      5497

Confusion Matrix:
[[1278  233   51]
 [ 189 1888  153]
 [ 106  165 1434]]


**ᴍᴏᴅᴇʟ ᴘᴇʀᴅɪᴄᴀᴛɪᴏɴ ᴠᴇʀꜱɪᴏɴ 3.0**

In [46]:

sample_text = ["I love this product!"]
sample_text = tokenizer.texts_to_sequences(sample_text)
sample_text = pad_sequences(sample_text, maxlen=MAX_SEQUENCE_LENGTH)

prediction = model3.predict(sample_text)
print("Prediction:", prediction)
print("Predicted label:", le.inverse_transform(np.argmax(prediction, axis=1)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Prediction: [[2.7029388e-04 9.8115420e-03 9.8991817e-01]]
Predicted label: ['positive']


**ᴍᴏᴅᴇʟ ᴛʀᴀɪɴɢ ᴠᴇʀꜱɪᴏɴ 4.0**

In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense

# Define the LSTM model
EMBEDDING_DIM = 100
MAX_VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 100

model4 = Sequential()
model4.add(Embedding(MAX_VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model4.add(SpatialDropout1D(0.2))
model4.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model4.add(Dense(num_classes, activation='softmax'))  # Number of classes for one-hot encoding

model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the Model
BATCH_SIZE = 64
EPOCHS = 8

history = model4.fit(X_train, y_train_one_hot, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test_one_hot), verbose=2)

Epoch 1/8


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


344/344 - 76s - 220ms/step - accuracy: 0.7344 - loss: 0.6382 - val_accuracy: 0.8305 - val_loss: 0.4600
Epoch 2/8
344/344 - 66s - 192ms/step - accuracy: 0.8468 - loss: 0.4079 - val_accuracy: 0.8430 - val_loss: 0.4243
Epoch 3/8
344/344 - 104s - 303ms/step - accuracy: 0.8713 - loss: 0.3468 - val_accuracy: 0.8406 - val_loss: 0.4326
Epoch 4/8
344/344 - 142s - 414ms/step - accuracy: 0.8844 - loss: 0.3111 - val_accuracy: 0.8448 - val_loss: 0.4377
Epoch 5/8
344/344 - 116s - 337ms/step - accuracy: 0.8942 - loss: 0.2887 - val_accuracy: 0.8350 - val_loss: 0.4465
Epoch 6/8
344/344 - 62s - 180ms/step - accuracy: 0.9020 - loss: 0.2689 - val_accuracy: 0.8405 - val_loss: 0.4522
Epoch 7/8
344/344 - 84s - 244ms/step - accuracy: 0.9071 - loss: 0.2518 - val_accuracy: 0.8423 - val_loss: 0.4698
Epoch 8/8
344/344 - 79s - 231ms/step - accuracy: 0.9139 - loss: 0.2366 - val_accuracy: 0.8375 - val_loss: 0.4928


**ᴍᴏᴅᴇʟ ᴇᴠᴀʟᴜᴀᴛɪᴏɴ ᴠᴇʀꜱɪᴏɴ 4.0**

In [48]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Predict class probabilities for X_test
y_pred_probs = model4.predict(X_test)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test_one_hot, axis=1)  # Ensure y_test is in one-hot encoded format

# Print classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['positive', 'negative', 'neutral']))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

172/172 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step
Classification Report:
              precision    recall  f1-score   support

    positive       0.84      0.79      0.82      1562
    negative       0.82      0.86      0.84      2230
     neutral       0.86      0.85      0.85      1705

    accuracy                           0.84      5497
   macro avg       0.84      0.83      0.84      5497
weighted avg       0.84      0.84      0.84      5497

Confusion Matrix:
[[1240  268   54]
 [ 131 1915  184]
 [ 100  156 1449]]


**ᴍᴏᴅᴇʟ ᴘᴇʀᴅɪᴄᴀᴛɪᴏɴ ᴠᴇʀꜱɪᴏɴ 4.0**

In [49]:

sample_text = ["I love this product!"]
sample_text = tokenizer.texts_to_sequences(sample_text)
sample_text = pad_sequences(sample_text, maxlen=MAX_SEQUENCE_LENGTH)

prediction = model4.predict(sample_text)
print("Prediction:", prediction)
print("Predicted label:", le.inverse_transform(np.argmax(prediction, axis=1)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Prediction: [[8.6013530e-04 1.8566947e-02 9.8057288e-01]]
Predicted label: ['positive']


**ᴍᴏᴅᴇʟ ᴛʀᴀɪɴɪɴɢ ᴠᴇʀꜱɪᴏɴ 5.0**

In [50]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense

# Define the LSTM model
EMBEDDING_DIM = 100
MAX_VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 100

model5 = Sequential()
model5.add(Embedding(MAX_VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model5.add(SpatialDropout1D(0.2))
model5.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model5.add(Dense(num_classes, activation='softmax'))  # Number of classes for one-hot encoding

model5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the Model
BATCH_SIZE = 64
EPOCHS = 9

history = model5.fit(X_train, y_train_one_hot, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test_one_hot), verbose=2)

Epoch 1/9


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


344/344 - 63s - 182ms/step - accuracy: 0.7378 - loss: 0.6339 - val_accuracy: 0.8303 - val_loss: 0.4540
Epoch 2/9
344/344 - 82s - 239ms/step - accuracy: 0.8458 - loss: 0.4064 - val_accuracy: 0.8459 - val_loss: 0.4225
Epoch 3/9
344/344 - 84s - 244ms/step - accuracy: 0.8690 - loss: 0.3486 - val_accuracy: 0.8405 - val_loss: 0.4266
Epoch 4/9
344/344 - 79s - 231ms/step - accuracy: 0.8857 - loss: 0.3110 - val_accuracy: 0.8410 - val_loss: 0.4428
Epoch 5/9
344/344 - 82s - 240ms/step - accuracy: 0.8951 - loss: 0.2856 - val_accuracy: 0.8314 - val_loss: 0.4532
Epoch 6/9
344/344 - 85s - 247ms/step - accuracy: 0.9017 - loss: 0.2685 - val_accuracy: 0.8335 - val_loss: 0.4772
Epoch 7/9
344/344 - 60s - 176ms/step - accuracy: 0.9093 - loss: 0.2492 - val_accuracy: 0.8325 - val_loss: 0.4787
Epoch 8/9
344/344 - 82s - 238ms/step - accuracy: 0.9133 - loss: 0.2326 - val_accuracy: 0.8432 - val_loss: 0.4903
Epoch 9/9
344/344 - 84s - 245ms/step - accuracy: 0.9191 - loss: 0.2191 - val_accuracy: 0.8363 - val_loss: 

**ᴍᴏᴅᴇʟ ᴇᴠᴀʟᴜᴀᴛɪᴏɴ ᴠᴇʀꜱɪᴏɴ 5.0**



In [51]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Predict class probabilities for X_test
y_pred_probs = model5.predict(X_test)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test_one_hot, axis=1)  # Ensure y_test is in one-hot encoded format

# Print classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['positive', 'negative', 'neutral']))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

172/172 ━━━━━━━━━━━━━━━━━━━━ 9s 50ms/step
Classification Report:
              precision    recall  f1-score   support

    positive       0.81      0.82      0.81      1562
    negative       0.82      0.85      0.84      2230
     neutral       0.88      0.83      0.86      1705

    accuracy                           0.84      5497
   macro avg       0.84      0.83      0.84      5497
weighted avg       0.84      0.84      0.84      5497

Confusion Matrix:
[[1278  238   46]
 [ 193 1898  139]
 [ 106  178 1421]]


**ᴍᴏᴅᴇʟ ᴘʀᴇᴅɪᴄᴀᴛɪᴏɴ ᴠᴇʀꜱɪᴏɴ 5.0**

In [54]:

sample_text = ["My Boyfriend encourages me"]
sample_text = tokenizer.texts_to_sequences(sample_text)
sample_text = pad_sequences(sample_text, maxlen=MAX_SEQUENCE_LENGTH)

prediction = model5.predict(sample_text)
print("Prediction:", prediction)
print("Predicted label:", le.inverse_transform(np.argmax(prediction, axis=1)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
Prediction: [[0.03135274 0.956878   0.01176931]]
Predicted label: ['neutral']


**ᴍᴏᴅᴇʟ ᴛʀᴀɪɴɪɴɢ ᴠᴇʀꜱɪᴏɴ 6.0**

In [53]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense

# Define the LSTM model
EMBEDDING_DIM = 100
MAX_VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 100

model6 = Sequential()
model6.add(Embedding(MAX_VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model6.add(SpatialDropout1D(0.2))
model6.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model6.add(Dense(num_classes, activation='softmax'))  # Number of classes for one-hot encoding

model6.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the Model
BATCH_SIZE = 64
EPOCHS = 10

history = model6.fit(X_train, y_train_one_hot, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test_one_hot), verbose=2)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


344/344 - 83s - 240ms/step - accuracy: 0.7364 - loss: 0.6367 - val_accuracy: 0.8306 - val_loss: 0.4570
Epoch 2/10
344/344 - 61s - 178ms/step - accuracy: 0.8478 - loss: 0.4060 - val_accuracy: 0.8465 - val_loss: 0.4239
Epoch 3/10
344/344 - 82s - 238ms/step - accuracy: 0.8698 - loss: 0.3469 - val_accuracy: 0.8417 - val_loss: 0.4229
Epoch 4/10
344/344 - 82s - 238ms/step - accuracy: 0.8842 - loss: 0.3120 - val_accuracy: 0.8425 - val_loss: 0.4308
Epoch 5/10
344/344 - 84s - 245ms/step - accuracy: 0.8938 - loss: 0.2865 - val_accuracy: 0.8365 - val_loss: 0.4556
Epoch 6/10
344/344 - 61s - 178ms/step - accuracy: 0.9019 - loss: 0.2668 - val_accuracy: 0.8448 - val_loss: 0.4705
Epoch 7/10
344/344 - 81s - 237ms/step - accuracy: 0.9083 - loss: 0.2508 - val_accuracy: 0.8332 - val_loss: 0.4755
Epoch 8/10
344/344 - 84s - 243ms/step - accuracy: 0.9120 - loss: 0.2354 - val_accuracy: 0.8355 - val_loss: 0.4937
Epoch 9/10
344/344 - 80s - 232ms/step - accuracy: 0.9164 - loss: 0.2205 - val_accuracy: 0.8357 - va

**ᴍᴏᴅᴇʟ ᴇᴠᴀʟᴜᴀᴛɪᴏɴ  ᴠᴇʀꜱɪᴏɴ 6.0**

In [55]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Predict class probabilities for X_test
y_pred_probs = model6.predict(X_test)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test_one_hot, axis=1)  # Ensure y_test is in one-hot encoded format

# Print classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['positive', 'negative', 'neutral']))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

172/172 ━━━━━━━━━━━━━━━━━━━━ 12s 69ms/step
Classification Report:
              precision    recall  f1-score   support

    positive       0.80      0.83      0.81      1562
    negative       0.83      0.83      0.83      2230
     neutral       0.87      0.84      0.86      1705

    accuracy                           0.83      5497
   macro avg       0.83      0.83      0.83      5497
weighted avg       0.83      0.83      0.83      5497

Confusion Matrix:
[[1294  218   50]
 [ 221 1853  156]
 [ 112  158 1435]]


**ᴍᴏᴅᴇʟ ᴘʀᴇᴅɪᴄᴀᴛɪᴏɴ ᴠᴇʀꜱɪᴏɴ 6.0**

In [ ]:

sample_text = ["I love this product!"]
sample_text = tokenizer.texts_to_sequences(sample_text)
sample_text = pad_sequences(sample_text, maxlen=MAX_SEQUENCE_LENGTH)

prediction = model6.predict(sample_text)
print("Prediction:", prediction)
print("Predicted label:", le.inverse_transform(np.argmax(prediction, axis=1)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Prediction: [[0.00218491 0.11231855 0.88549656]]
Predicted label: ['positive']


**8. ꜱᴀᴠɪɴɢ ᴍᴏᴅᴇʟ 4**

In [56]:
# Save the trained model3
model3.save('sentiment_analysis_model3.h5')


In [74]:
from tensorflow.keras.models import load_model

# Assuming 'model' is your trained model
model3.save('sentiment_analysis_model.h5')



In [72]:
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer

# Assuming 'tokenizer' is your Tokenizer object
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [12]:
from joblib import dump, load

# Save the tokenizer
dump(tokenizer, 'tokenizer.joblib')

['tokenizer.joblib']